In [ ]:
import ujson
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
import psutil as ps
import datetime
import seaborn as sns

LOGS = './logs/'
FILENAME = LOGS + os.listdir(LOGS)[-1]
UPDATE_TIME = 10  # seconds

def load(filename):
    with open(filename, mode='r') as file:
        raw = ujson.load(file)
        data = {int(timestamp): ps._common.snetio(*fields) for timestamp, fields in raw.items()}
    return data

raw = load(FILENAME)

In [ ]:
data = pd.DataFrame.from_dict(data=raw, orient='index')
data = data[['bytes_sent', 'bytes_recv']]
data = data.apply(lambda x: x/1024/1024)
data.sort_index(inplace=True)
data.index = data.index.map(datetime.datetime.fromtimestamp)

In [ ]:
data.plot()

In [ ]:
data = data.resample('{}s'.format(UPDATE_TIME)).sum()
data['download_speed'] = data['bytes_recv'].diff().apply(lambda x: x/UPDATE_TIME*8)  # megabits/second
data['upload_speed'] =   data['bytes_sent'].diff().apply(lambda x: x/UPDATE_TIME*8)  # megabits/second

In [ ]:
data[['download_speed', 'upload_speed']].plot()